In [8]:
import numpy as np
from sklearn.neighbors import KernelDensity
import torch
import sys
import os
import pickle
from get_data import get_dataloader # noqa

In [2]:
def estimate_density(data, bandwidth=1.0, kernel='gaussian'):
    kde = KernelDensity(kernel=kernel, bandwidth=bandwidth)
    kde.fit(data)
    return kde

def get_log_density(kde_model, input_data):
    """
    Get the probability density of an arbitrary 64-dimensional input.

    Parameters:
    kde_model: Trained KernelDensity model.
    input_data (np.ndarray): A single 64-dimensional input, or multiple inputs (n_samples, 64).

    Returns:
    density (np.ndarray): The estimated probability density for the input.
    """
    
    # Ensure input data has the correct number of dimensions
    if input_data.ndim == 1:
        input_data = input_data.reshape(1, -1)

    # Estimate log density for the input
    log_density = kde_model.score_samples(input_data)

    # # Convert log density to regular density
    # density = np.exp(log_density)

    return log_density

In [24]:
kdes = {}
label_pdf = {}
with open('experiments/DATA_redundancy.pickle', 'rb') as f:
    data = pickle.load(f)['train']
    for modality in {'0', '1'}:
        data_mod = data[modality]
        kde = estimate_density(data_mod)
        kdes[modality] = kde
    

    label_count = np.bincount(data['label'].flatten())
    # convert label_count to log distribution
    label_probs = np.log(label_count / np.sum(label_count))
    label_pdf = {i: label_probs[i] for i in range(len(label_probs))}


In [26]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
encoders = {}
heads = {}

saved_model = 'experiments/redundancy/redundancy_unimodal'

for modality in {'0', '1'}:
    saved_encoder = saved_model + '_encoder_{}.pt'.format(modality)
    saved_head = saved_model + '_head_{}.pt'.format(modality)
    encoders[modality] = torch.load(saved_encoder).to(device)
    heads[modality] = torch.load(saved_head).to(device)

print(encoders, heads)

FileNotFoundError: [Errno 2] No such file or directory: 'experiments/redundancy/redundancy_encoder.pt'